# August 8, 2022 Working Map

In [1]:
import numpy as np
import pandas as pd

import bokeh.io

import panel as pn
import datetime as dt

from bokeh.tile_providers import CARTODBPOSITRON, get_provider

from bokeh.models import Div, HoverTool, CustomJS, ColumnDataSource, Select, CDSView, CustomJSHover, CustomJSFilter

tile_provider = get_provider(CARTODBPOSITRON)

bokeh.io.output_notebook()
pn.extension()

Loading BokehJS ...

In [2]:
pn.config.raw_css  = [
    """
        /* div table styling */
        table, th, td {
          border: 1px solid black;
          border-collapse: collapse;
        }
        
        .choices__inner div[data-value=Dairy] {
          background-color: #1f77b4 !important;
        }
        .choices__inner div[data-value=Grain] {
          background-color: #ff7f0e !important;
        }
        .choices__inner div[data-value=Control] {
          background-color: #aec7e8 !important;
        }
        .choices__inner div[data-value=Bean] {
          background-color: #ffbb78 !important;
        }
        .choices__inner div[data-value=Drink] {
          background-color: #2ca02c !important;
        }
        .choices__inner div[data-value=Vegetable] {
          background-color: #98df8a !important;
        }
        .choices__inner div[data-value=Fruit] {
          background-color: #d62728 !important;
        }
        .choices__inner div[data-value=Fish] {
          background-color: #ff9896 !important;
        }
        .choices__inner div[data-value=Vinegar] {
          background-color: #9467bd !important;
        }
        .choices__inner div[data-value=Shellfish] {
          background-color: #c5b0d5 !important;
        }
        .choices__inner div[data-value=Intestine] {
          background-color: #8c564b !important;
        }
        .choices__inner div[data-value=Meat] {
          background-color: #c49c94 !important;
        }
        .choices__inner div[data-value=Crustacean] {
          background-color: #e377c2 !important;
        }
        .choices__inner div[data-value=Unclear] {
          background-color: #f7b6d2 !important;
        }
    """
]

# Load and prep data

In [3]:
# Helper fxn to set coords right
def mercator_creator(df, lat='lat', lon='lon'):
    # from https://www.youtube.com/watch?v=BojxegBh9_4
    
    k = 6378137
    df['x'] = k * np.radians(df[lon])
    df['y'] = np.log(np.tan((90 + df[lat]) * np.pi / 360)) * k
    
    return df

In [4]:
# Load in the metadata
ferm_locations = pd.read_csv("./data/ferm_updated.csv")

# Split the Lat-Long column into two columns, Lat and Long
ferm_locations[['Lat', 'Long']] = ferm_locations['Lat-Long'].str.split(', ', 1, expand=True)

# Create a new dataframe only if the point has a valid Lat and Long
ferm_locations_final = ferm_locations[ferm_locations['Lat'].notna()].copy()

# Convert from a string, and put in columns called Latitude and Longitude
ferm_locations_final.loc[:,'Latitude1'] = pd.to_numeric(ferm_locations_final['Lat'],errors='coerce')
ferm_locations_final.loc[:,'Longitude1'] = pd.to_numeric(ferm_locations_final['Long'],errors='coerce')

# Add x, y for lat long
ferm_locations_final = mercator_creator(ferm_locations_final, lat='Latitude1', lon='Longitude1')

class_options = list(ferm_locations_final.Class.unique())
country_options = list(ferm_locations_final.Country_obtained.unique())

ferm_locations_final.Acquisition_date = pd.to_datetime(ferm_locations_final.Acquisition_date)
ferm_locations_final.Extract_date = pd.to_datetime(ferm_locations_final.Extract_date)

In [5]:
abundance = pd.read_csv('./data/level-7.csv')
abundance = abundance.filter(regex='k__|index|;__')
cols = abundance.columns[~abundance.columns.str.contains('index')]
abundance[cols] = abundance[cols].div(abundance[cols].sum(axis=1), axis=0)

abundance = abundance.transpose()
abundance.columns = abundance.iloc[0]
abundance = abundance.drop(abundance.index[0])
abundance.columns.name = ''

# Convert to numeric
cols = abundance.columns
abundance[cols] = abundance[cols].apply(pd.to_numeric, errors='coerce')

abundance = abundance.transpose()

# Create a Column called 'Sample_ID'
abundance = abundance.reset_index().rename(columns={abundance.index.name:'Sample_ID'})

abundance.head()

,Sample_ID,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides;s__,k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria;__,k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;__,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides;__,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Pasteurellales;f__Pasteurellaceae;g__Haemophilus;s__parainfluenzae,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides;s__uniformis,k__Bacteria;p__Firmicutes;c__Bacilli;__;__;__;__,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o__Fusobacteriales;f__Fusobacteriaceae;g__Fusobacterium;s__,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Prevotellaceae;g__Prevotella;s__melaninogenica,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Pseudomonadales;f__Pseudomonadaceae;g__Pseudomonas;s__veronii,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Faecalibacterium;s__prausnitzii,k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria;s__subflava
0,LNM1,0.435416,0.000958,0.0,0.153890,0.000000,0.229398,0.000000,0.005941,0.0,0.0,0.174396,0.0
1,LNM2,0.469008,0.000000,0.0,0.275920,0.000000,0.121525,0.000000,0.000000,0.0,0.0,0.133546,0.0
2,LNM3,0.477214,0.000000,0.0,0.230026,0.000986,0.190176,0.001184,0.000000,0.0,0.0,0.100414,0.0
3,LNM4,0.418389,0.000000,0.0,0.221845,0.000000,0.280469,0.002160,0.000000,0.0,0.0,0.077137,0.0
4,LNM5,0.515295,0.000000,0.0,0.179922,0.000000,0.147664,0.000000,0.000000,0.0,0.0,0.157119,0.0


In [6]:
test2=abundance.loc[abundance.Sample_ID == 'LNM1']
test2

,Sample_ID,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides;s__,k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria;__,k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;__,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides;__,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Pasteurellales;f__Pasteurellaceae;g__Haemophilus;s__parainfluenzae,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides;s__uniformis,k__Bacteria;p__Firmicutes;c__Bacilli;__;__;__;__,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o__Fusobacteriales;f__Fusobacteriaceae;g__Fusobacterium;s__,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Prevotellaceae;g__Prevotella;s__melaninogenica,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Pseudomonadales;f__Pseudomonadaceae;g__Pseudomonas;s__veronii,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Faecalibacterium;s__prausnitzii,k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria;s__subflava
0,LNM1,0.435416,0.000958,0.0,0.15389,0.0,0.229398,0.0,0.005941,0.0,0.0,0.174396,0.0


# Bokeh (Faster)

In [7]:
colors = bokeh.palettes.Category20[15]

color_map = dict(zip(class_options, colors))

ferm_locations_final['color'] = ferm_locations_final['Class'].map(color_map)

In [8]:
ferm_locations_final['url']  = ["https://images.pexels.com/photos/326875/pexels-photo-326875.jpeg?w=800&h=600&auto=compress&cs=tinysrgb"] * len(ferm_locations_final)

In [9]:
# Set up the plot
p = bokeh.plotting.figure(
        frame_height=300,
        frame_width=800,
        x_axis_label="Latitude",
        y_axis_label="Longitude",
        tools =['wheel_zoom', 'pan', 'reset', 'tap'],
        x_axis_type = 'mercator',
        y_axis_type = 'mercator',
        x_range = [ferm_locations_final.x.min()-100000, ferm_locations_final.x.max() + 100000],
        y_range = [ferm_locations_final.y.min()-100000, ferm_locations_final.y.max() + 100000],
        title = 'Cultures Across Cultures Fermentation Explorer'
    )

p.xaxis.axis_label_text_font_size = "10pt"
p.yaxis.axis_label_text_font_size = "10pt"
p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.title.text_font_style = "bold"
p.title.text_font_size = "12pt"
# p.title.align = "center"

p.xaxis.major_label_text_font_size = "10pt"
p.yaxis.major_label_text_font_size = "10pt"

p.outline_line_color = 'black'
p.outline_line_width = 2

# change just some things about the y-grid
p.ygrid.grid_line_alpha = 0.7
p.xgrid.grid_line_alpha =0.7

p.xaxis.major_tick_line_width = 2

p.yaxis.major_tick_line_width = 2

p.axis.major_tick_out = 8
p.axis.minor_tick_in = -3
p.axis.minor_tick_out = 8

p.toolbar.logo = 'grey'

p.toolbar_location = 'above'
p.add_tile(tile_provider)
p.add_tools(bokeh.models.BoxZoomTool(match_aspect=True))

In [10]:
# point available
cds = bokeh.models.ColumnDataSource(
    {
        "x": ferm_locations_final["x"],
        "y": ferm_locations_final["y"],
        "color": ferm_locations_final['color'],
        "Country_obtained": ferm_locations_final['Country_obtained'],
        "Class": ferm_locations_final['Class'],
        "Sample_ID": ferm_locations_final['Sample_ID'],
        "Type": ferm_locations_final['Type'],
        "url": ferm_locations_final['url'],
    }
)

# points selected
chosen_cds = bokeh.models.ColumnDataSource(
    {
        "x": ferm_locations_final["x"],
        "y": ferm_locations_final["y"],
        "color": ferm_locations_final['color'],
        "Country_obtained": ferm_locations_final['Country_obtained'],
        "Class": ferm_locations_final['Class'],
        "Sample_ID": ferm_locations_final['Sample_ID'],
        "Type": ferm_locations_final['Type'],
        "url": ferm_locations_final['url'],
    }
)

In [11]:
hover = bokeh.models.HoverTool()

hover.tooltips = """
    <div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@Type</span>
            <span style="font-size: 15px; color: #966;">@Class</span>
        </div>
        <div>
            <span><b>@Sample_ID</b></span>
        </div>
        <hr>
    </div>
"""
p.add_tools(hover)

In [12]:
circle = p.circle(
    source=chosen_cds, x="x", y="y", fill_color="color", line_color="color", size = 7
)

In [13]:
# points available
barplotcds_bact = bokeh.models.ColumnDataSource(abundance)

In [14]:
# JavaScript code for the callback stored as a string
jscode = """
let selected_class = class_choice.value;
let selected_country = country_choice.value;

let point_visible = source_visible.data;
let point_available = source_available.data;

point_visible.x = []
point_visible.y = []
point_visible.color = []
point_visible.Country_obtained = []
point_visible.Class = []
point_visible.Sample_ID = []
point_visible.Type = []
point_visible.url = []

for (let i = 0; i < point_available.x.length; i++) {
    if (selected_class.includes(point_available.Class[i]) && selected_country.includes(point_available.Country_obtained[i])) {
            point_visible.x.push(point_available.x[i]);
        point_visible.y.push(point_available.y[i]);
        point_visible.color.push(point_available.color[i]);
        point_visible.Country_obtained.push(point_available.Country_obtained[i]);
        point_visible.Class.push(point_available.Class[i]);
        point_visible.Sample_ID.push(point_available.Sample_ID[i]);
        point_visible.Type.push(point_available.Type[i]);
        point_visible.url.push(point_available.url[i]);
    }
}

source_visible.change.emit(); 
"""

In [15]:
sp = test2.drop('Sample_ID', axis=1).columns
sample = ['LNM1']

pal = list(bokeh.palettes.Set2[8])
mult_factor = np.floor(len(sp) / len(pal))

# Expand colors until they are at least the right size
colors = pal * int(np.ceil(len(sp) / len(pal)))
# Trim the excess
colors = colors[:len(sp)]

barplot = bokeh.plotting.figure(title="Bacterial",toolbar_location='right', width = 200, height = 380,
           y_axis_label = 'Relative Abundance', x_range=sample, tools = ['pan', 'wheel_zoom', 'reset'], y_range=(0, 1))

barplot.vbar_stack(sp, x='Sample_ID', width=0.8, source=barplotcds_bact, color=colors)
barplot.x_range.factors = ['LNM1']

t = """
<div id = "abundance", style = "width:200px;">
    <b>Relative Abundance:</b> @$name
    <hr>
    $name{custom} <br>
</div>
"""

tax_custom = CustomJSHover(code="""
    const myString = special_vars.name.split(";");
  
    var myStringSplit = [];

    var dict = {
      "k": "Kingdom:",
      "p": "Phylum:",
      "c": "Class:",
      "o": "Order:",
      "f": "Family:",
      "g": "Genus:",
      "s": "Species:",
    }
    
    myString.forEach(function(element, i) {
            var currElement = element.split("__");

            if (currElement[0] != "") {
                myStringSplit.push(`<b>${dict[currElement[0]]}</b> ${currElement[1]}`);
            }
        });
    
    return myStringSplit.join('<br>');
""")

barplot.add_tools(HoverTool(
    tooltips=t,
    formatters={'$name': tax_custom}
))

barplot.xaxis.axis_label_text_font_size = "10pt"
barplot.yaxis.axis_label_text_font_size = "10pt"
barplot.xaxis.axis_label_text_font_style = "normal"
barplot.yaxis.axis_label_text_font_style = "normal"

barplot.title.text_font_style = "bold"
barplot.title.text_font_size = "12pt"

barplot.xaxis.major_label_text_font_size = "10pt"
barplot.yaxis.major_label_text_font_size = "10pt"

barplot.ygrid.grid_line_alpha = 0.7
barplot.xgrid.grid_line_alpha = 0

barplot.xaxis.major_tick_line_width = 2
barplot.yaxis.major_tick_line_width = 2

barplot.outline_line_color = None

barplot.toolbar.logo = None

In [16]:
bokeh.io.show(barplot)

In [17]:
div = Div(text="""<img src="${curr_url}" width = 300/>""")

datatable_cds = ColumnDataSource()

# JavaScript code for the callback stored as a string
jscode_sample = CustomJS(args=dict(barplot=barplot, div=div, source_visible=chosen_cds, barplotcds_bact=barplotcds_bact), code="""
let selected_sample = cb_obj.value;
let point_visible = source_visible.data;

for (let i = 0; i < point_visible.Sample_ID.length; i++) {
    if (point_visible.Sample_ID[i] == selected_sample) {
        let curr_url = point_visible.url[i];
        div.text = '<img src="${curr_url}" width = 300/>'
    }
}
""")

sample_choice = bokeh.models.Select(title="View photo for sample:",value = 'LNM1', options=['LNM1'])

sample_choice.js_on_change('value', jscode_sample)

In [18]:
cols_to_show = [
    bokeh.models.TableColumn(field = 'Sample_ID', title = 'Sample_ID'),
    bokeh.models.TableColumn(field = 'Type', title = 'Type'),
    bokeh.models.TableColumn(field = 'Class', title = 'Class'),
    bokeh.models.TableColumn(field = 'Country_obtained', title = 'Country_obtained'),
]

source_tapped = ColumnDataSource(data=chosen_cds.data)

datatable_display = bokeh.models.DataTable(source=source_tapped, columns = cols_to_show)

In [19]:
taptool = p.select(type=bokeh.models.TapTool)

taptool.callback = CustomJS(args=dict(source_tapped=source_tapped,datatable_display=datatable_display, sample_choice = sample_choice, barplot = barplot, source_available=cds, source_visible=chosen_cds, barplotcds_bact=barplotcds_bact), code="""
    let samples = []; 
        
    source_tapped.data.Country_obtained = []
    source_tapped.data.Class = []
    source_tapped.data.Sample_ID = []
    source_tapped.data.Type = []

    
    for (let i = 0; i < source_visible.selected.indices.length; i++) {
            let curr = source_visible.selected.indices[i];
            console.log(curr);
            
            samples.push(source_visible.data.Sample_ID[curr]);
            
            source_tapped.data.Country_obtained.push(source_visible.data.Country_obtained[curr]);
            source_tapped.data.Class.push(source_visible.data.Class[curr]);
            source_tapped.data.Sample_ID.push(source_visible.data.Sample_ID[curr]);
            source_tapped.data.Type.push(source_visible.data.Type[curr]);
        }
        
    barplot.x_range.factors = samples;
    barplot.width = 200 + samples.length * 50;
    sample_choice.options = samples;
    
    source_tapped.change.emit(); 


""")

In [20]:
country_choice = pn.widgets.MultiChoice(name='Choose Countries:', value=country_options,
    options=country_options, solid=False, sizing_mode='stretch_width')

class_choice = pn.widgets.MultiChoice(name='Choose Ferment Classes:', value=class_options,
    options=class_options, sizing_mode='stretch_width')

In [21]:
class_choice.jscallback(value=jscode, args=dict(source_available=cds, source_visible=chosen_cds, class_choice=class_choice, country_choice=country_choice))

Callback(args={'source_available': ColumnDataSource(id='1045', ...), 'source_visible': ColumnDataSource(id='1046', ...), 'class_choice': MultiChoice(name='Choose Ferment Classes:', options=['Dairy', 'Control', ...], sizing_mode='stretch_width', value=['Dairy', 'Control', ...]), 'country_choice': MultiChoice(name='Choose Countries:', options=['France', 'Singapore', ...], sizing_mode='stretch_width', solid=False, value=['France', 'Singapore', ...])}, code={'value': '\nlet selected_class = class_choice.value;\nlet selected_country = country_choice.value;\n\nlet point_visible = source_visible.data;\nlet point_available = source_available.data;\n\npoint_visible.x = []\npoint_visible.y = []\npoint_visible.color = []\npoint_visible.Country_obtained = []\npoint_visible.Class = []\npoint_visible.Sample_ID = []\npoint_visible.Type = []\npoint_visible.url = []\n\nfor (let i = 0; i < point_available.x.length; i++) {\n    if (selected_class.includes(point_available.Class[i]) && selected_country.inc

In [22]:
country_choice.jscallback(value=jscode, args=dict(source_available=cds, source_visible=chosen_cds, class_choice=class_choice, country_choice=country_choice))

Callback(args={'source_available': ColumnDataSource(id='1045', ...), 'source_visible': ColumnDataSource(id='1046', ...), 'class_choice': MultiChoice(name='Choose Ferment Classes:', options=['Dairy', 'Control', ...], sizing_mode='stretch_width', value=['Dairy', 'Control', ...]), 'country_choice': MultiChoice(name='Choose Countries:', options=['France', 'Singapore', ...], sizing_mode='stretch_width', solid=False, value=['France', 'Singapore', ...])}, code={'value': '\nlet selected_class = class_choice.value;\nlet selected_country = country_choice.value;\n\nlet point_visible = source_visible.data;\nlet point_available = source_available.data;\n\npoint_visible.x = []\npoint_visible.y = []\npoint_visible.color = []\npoint_visible.Country_obtained = []\npoint_visible.Class = []\npoint_visible.Sample_ID = []\npoint_visible.Type = []\npoint_visible.url = []\n\nfor (let i = 0; i < point_available.x.length; i++) {\n    if (selected_class.includes(point_available.Class[i]) && selected_country.inc

In [23]:
p_pane = pn.pane.Bokeh(p)
p_div = pn.pane.Bokeh(div)

map_panel = pn.Column(
    pn.Row(p, pn.Column(pn.Spacer(height = 10), country_choice, class_choice)),
    pn.layout.Divider(margin=(-5, 0, 0, 0)),
    pn.Row(pn.Spacer(width = 50), pn.Column(datatable_display),pn.Spacer(width = 50), pn.Column(barplot))
)

map_panel

Column
    [0] Row
        [0] Bokeh(Figure)
        [1] Column
            [0] Spacer(height=10)
            [1] MultiChoice(name='Choose Countries:', options=['France', 'Singapore', ...], sizing_mode='stretch_width', solid=False, value=['France', 'Singapore', ...])
            [2] MultiChoice(name='Choose Ferment Classes:', options=['Dairy', 'Control', ...], sizing_mode='stretch_width', value=['Dairy', 'Control', ...])
    [1] Divider(margin=(-5, 0, 0, 0))
    [2] Row
        [0] Spacer(width=50)
        [1] Column
            [0] Bokeh(DataTable)
        [2] Spacer(width=50)
        [3] Column
            [0] Bokeh(Figure)

In [25]:
map_panel.save('august8_CURRENT_PANEL.html', embed=True)
